# Insurance: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using an insurance analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Claims Processing and Risk Assessment

We'll analyze insurance claims and policy data. Our clustering strategy will optimize for:

- **Policyholder-specific queries**: Fast lookups by customer ID
- **Time-based analysis**: Efficient filtering by claim and policy dates
- **Risk patterns**: Quick aggregation by claim type and risk scores

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create insurance catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS insurance")

spark.sql("CREATE SCHEMA IF NOT EXISTS insurance.analytics")

print("Insurance catalog and analytics schema created successfully!")

Insurance catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `insurance_claims` table will store:

- **customer_id**: Unique policyholder identifier
- **claim_date**: Date claim was filed
- **policy_type**: Type of insurance (Auto, Home, Health, etc.)
- **claim_amount**: Claim payout amount
- **risk_score**: Customer risk assessment (1-100)
- **processing_time**: Days to process claim
- **claim_status**: Approved, Denied, Pending

### Clustering Strategy

We'll cluster by `customer_id` and `claim_date` because:

- **customer_id**: Policyholders often file multiple claims, grouping their insurance history together
- **claim_date**: Time-based queries are critical for fraud detection, seasonal analysis, and regulatory reporting
- This combination optimizes for both customer risk profiling and temporal claims analysis

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS insurance.analytics.insurance_claims (

    customer_id STRING,

    claim_date DATE,

    policy_type STRING,

    claim_amount DECIMAL(10,2),

    risk_score INT,

    processing_time INT,

    claim_status STRING

)

USING DELTA

CLUSTER BY (customer_id, claim_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on customer_id and claim_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on customer_id and claim_date.


## Step 3: Generate Insurance Sample Data

### Data Generation Strategy

We'll create realistic insurance claims data including:

- **8,000 customers** with multiple claims over time
- **Policy types**: Auto, Home, Health, Life, Property
- **Realistic claim patterns**: Seasonal variations, claim frequencies, processing times
- **Risk scoring**: Customer risk assessment and fraud indicators

### Why This Data Pattern?

This data simulates real insurance scenarios where:

- Customer claims history affects risk assessment
- Seasonal patterns impact claim volumes
- Processing efficiency affects customer satisfaction
- Fraud detection requires pattern analysis
- Regulatory reporting demands temporal analysis

In [ ]:
# Generate sample insurance claims data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define insurance data constants

POLICY_TYPES = ['Auto', 'Home', 'Health', 'Life', 'Property']

CLAIM_STATUSES = ['Approved', 'Denied', 'Pending']

# Base claim parameters by policy type

CLAIM_PARAMS = {

    'Auto': {'avg_claim': 3500, 'frequency': 3, 'processing_days': 14},

    'Home': {'avg_claim': 8500, 'frequency': 1, 'processing_days': 21},

    'Health': {'avg_claim': 1200, 'frequency': 8, 'processing_days': 7},

    'Life': {'avg_claim': 25000, 'frequency': 0.5, 'processing_days': 30},

    'Property': {'avg_claim': 15000, 'frequency': 1.5, 'processing_days': 18}

}


# Generate insurance claims records

claims_data = []

base_date = datetime(2024, 1, 1)


# Create 8,000 customers with 1-12 claims each (based on frequency)

for customer_num in range(1, 8001):

    customer_id = f"CUST{customer_num:06d}"
    
    # Assign a primary policy type for this customer

    primary_policy = random.choice(POLICY_TYPES)

    params = CLAIM_PARAMS[primary_policy]
    
    # Determine number of claims based on frequency (some customers have no claims)

    if random.random() < 0.3:  # 30% of customers have no claims

        num_claims = 0

    else:

        num_claims = max(1, int(random.gauss(params['frequency'], params['frequency'] * 0.5)))
        num_claims = min(num_claims, 12)  # Cap at 12 claims
    
    # Generate claims

    for i in range(num_claims):

        # Spread claims over 12 months

        days_offset = random.randint(0, 365)

        claim_date = base_date + timedelta(days=days_offset)
        
        # Sometimes use different policy types for the same customer

        if random.random() < 0.2:

            policy_type = random.choice(POLICY_TYPES)

            params = CLAIM_PARAMS[policy_type]

        else:

            policy_type = primary_policy
        
        # Calculate claim amount with variation

        amount_variation = random.uniform(0.1, 3.0)

        claim_amount = round(params['avg_claim'] * amount_variation, 2)
        
        # Risk score (higher for larger/frequent claims)

        base_risk = random.randint(20, 80)

        risk_adjustment = min(20, int(claim_amount / 1000))  # Higher amounts increase risk

        risk_score = min(100, base_risk + risk_adjustment)
        
        # Processing time (varies by claim type and some randomness)

        time_variation = random.uniform(0.5, 2.0)

        processing_time = max(1, int(params['processing_days'] * time_variation))
        
        # Claim status (most approved, some denied, few pending)

        status_weights = [0.75, 0.15, 0.10]  # Approved, Denied, Pending

        claim_status = random.choices(CLAIM_STATUSES, weights=status_weights)[0]
        
        claims_data.append({

            "customer_id": customer_id,

            "claim_date": claim_date.date(),

            "policy_type": policy_type,

            "claim_amount": claim_amount,

            "risk_score": risk_score,

            "processing_time": processing_time,

            "claim_status": claim_status

        })



print(f"Generated {len(claims_data)} insurance claims records")

print("Sample record:", claims_data[0])

Generated 15000 insurance claims records
Sample record: {'customer_id': 'CUST000001', 'claim_date': datetime.date(2024, 8, 23), 'policy_type': 'Home', 'claim_amount': 5562.56, 'risk_score': 35, 'processing_time': 26, 'claim_status': 'Approved'}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_claims = spark.createDataFrame(claims_data)


# Display schema and sample data

print("DataFrame Schema:")

df_claims.printSchema()



print("\nSample Data:")

df_claims.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (customer_id, claim_date) will automatically optimize the data layout

df_claims.write.mode("overwrite").saveAsTable("insurance.analytics.insurance_claims")


print(f"\nSuccessfully inserted {df_claims.count()} records into insurance.analytics.insurance_claims")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- claim_amount: double (nullable = true)
 |-- claim_date: date (nullable = true)
 |-- claim_status: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- policy_type: string (nullable = true)
 |-- processing_time: long (nullable = true)
 |-- risk_score: long (nullable = true)


Sample Data:


+------------+----------+------------+-----------+-----------+---------------+----------+
|claim_amount|claim_date|claim_status|customer_id|policy_type|processing_time|risk_score|
+------------+----------+------------+-----------+-----------+---------------+----------+
|     5562.56|2024-08-23|    Approved| CUST000001|       Home|             26|        35|
|     6011.12|2024-08-26|      Denied| CUST000001|     Health|             24|        76|
|    23118.44|2024-08-03|    Approved| CUST000002|       Home|             34|        55|
|     30107.2|2024-04-25|    Approved| CUST000003|       Life|             31|        47|
|     2186.86|2024-01-04|    Approved| CUST000004|     Health|              5|        54|
+------------+----------+------------+-----------+-----------+---------------+----------+
only showing top 5 rows




Successfully inserted 15000 records into insurance.analytics.insurance_claims
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Customer claims history** (clustered by customer_id)
2. **Time-based claims analysis** (clustered by claim_date)
3. **Combined customer + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Customer claims history - benefits from customer_id clustering

print("=== Query 1: Customer Claims History ===")

customer_history = spark.sql("""

SELECT customer_id, claim_date, policy_type, claim_amount, claim_status

FROM insurance.analytics.insurance_claims

WHERE customer_id = 'CUST000001'

ORDER BY claim_date DESC

""")



customer_history.show()

print(f"Records found: {customer_history.count()}")



# Query 2: Time-based high-value claims analysis - benefits from claim_date clustering

print("\n=== Query 2: Recent High-Value Claims ===")

high_value_claims = spark.sql("""

SELECT claim_date, customer_id, policy_type, claim_amount, risk_score

FROM insurance.analytics.insurance_claims

WHERE claim_date >= '2024-06-01' AND claim_amount > 10000

ORDER BY claim_amount DESC, claim_date DESC

""")



high_value_claims.show()

print(f"High-value claims found: {high_value_claims.count()}")



# Query 3: Combined customer + time query - optimal for our clustering strategy

print("\n=== Query 3: Customer Claims Trends ===")

claims_trends = spark.sql("""

SELECT customer_id, claim_date, policy_type, claim_amount, risk_score

FROM insurance.analytics.insurance_claims

WHERE customer_id LIKE 'CUST000%' AND claim_date >= '2024-04-01'

ORDER BY customer_id, claim_date

""")



claims_trends.show()

print(f"Claims trend records found: {claims_trends.count()}")

=== Query 1: Customer Claims History ===


+-----------+----------+-----------+------------+------------+
|customer_id|claim_date|policy_type|claim_amount|claim_status|
+-----------+----------+-----------+------------+------------+
| CUST000001|2024-08-26|     Health|     6011.12|      Denied|
| CUST000001|2024-08-23|       Home|     5562.56|    Approved|
+-----------+----------+-----------+------------+------------+



Records found: 2

=== Query 2: Recent High-Value Claims ===


+----------+-----------+-----------+------------+----------+
|claim_date|customer_id|policy_type|claim_amount|risk_score|
+----------+-----------+-----------+------------+----------+
|2024-12-29| CUST005908|     Health|    74916.03|        96|
|2024-07-23| CUST007016|     Health|    74900.02|        52|
|2024-10-18| CUST001581|       Life|     74895.3|        80|
|2024-12-15| CUST004733|       Life|     74883.6|        97|
|2024-07-25| CUST002601|       Life|    74874.85|        57|
|2024-11-17| CUST005594|       Life|    74829.33|        57|
|2024-12-24| CUST005524|       Life|    74818.84|        82|
|2024-10-16| CUST001368|     Health|    74812.53|        68|
|2024-10-18| CUST005266|       Life|    74701.96|        88|
|2024-07-15| CUST001375|       Life|    74683.53|        71|
|2024-06-04| CUST007676|       Life|    74576.77|        45|
|2024-06-13| CUST004179|     Health|    74573.16|        55|
|2024-07-05| CUST005762|       Life|    74488.06|        91|
|2024-06-25| CUST005196|

High-value claims found: 3176

=== Query 3: Customer Claims Trends ===


+-----------+----------+-----------+------------+----------+
|customer_id|claim_date|policy_type|claim_amount|risk_score|
+-----------+----------+-----------+------------+----------+
| CUST000001|2024-08-23|       Home|     5562.56|        35|
| CUST000001|2024-08-26|     Health|     6011.12|        76|
| CUST000002|2024-08-03|       Home|    23118.44|        55|
| CUST000003|2024-04-25|       Life|     30107.2|        47|
| CUST000004|2024-06-17|     Health|     3418.76|        27|
| CUST000004|2024-07-13|   Property|      5252.1|        78|
| CUST000004|2024-09-21|     Health|     4055.38|        34|
| CUST000004|2024-12-21|     Health|     3026.38|        73|
| CUST000012|2024-07-10|       Auto|     5113.28|        45|
| CUST000014|2024-05-20|       Auto|     5076.39|        82|
| CUST000014|2024-07-18|       Auto|     7187.73|        30|
| CUST000015|2024-04-24|   Property|    10582.94|        78|
| CUST000015|2024-07-18|   Property|    20606.38|        77|
| CUST000016|2024-12-09|

Claims trend records found: 1340


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the insurance insights possible with this optimized structure.

### Key Analytics

- **Customer risk profiling** and claims frequency analysis
- **Policy performance** and loss ratio calculations
- **Claims processing efficiency** and operational metrics
- **Fraud detection patterns** and risk scoring effectiveness

In [ ]:
# Analyze clustering effectiveness and insurance insights


# Customer risk analysis

print("=== Customer Risk Analysis ===")

customer_risk = spark.sql("""

SELECT customer_id, COUNT(*) as total_claims,

       ROUND(SUM(claim_amount), 2) as total_claimed,

       ROUND(AVG(claim_amount), 2) as avg_claim_amount,

       ROUND(AVG(risk_score), 2) as avg_risk_score,

       ROUND(AVG(processing_time), 2) as avg_processing_days

FROM insurance.analytics.insurance_claims

GROUP BY customer_id

ORDER BY total_claimed DESC

LIMIT 10

""")



customer_risk.show()


# Policy type performance

print("\n=== Policy Type Performance ===")

policy_performance = spark.sql("""

SELECT policy_type, COUNT(*) as total_claims,

       ROUND(SUM(claim_amount), 2) as total_payout,

       ROUND(AVG(claim_amount), 2) as avg_claim_amount,

       ROUND(AVG(processing_time), 2) as avg_processing_days,

       COUNT(DISTINCT customer_id) as affected_customers

FROM insurance.analytics.insurance_claims

GROUP BY policy_type

ORDER BY total_payout DESC

""")



policy_performance.show()


# Claims processing efficiency

print("\n=== Claims Processing Efficiency ===")

processing_efficiency = spark.sql("""

SELECT 

    CASE 

        WHEN processing_time <= 7 THEN 'Fast (1-7 days)'

        WHEN processing_time <= 14 THEN 'Normal (8-14 days)'

        WHEN processing_time <= 21 THEN 'Slow (15-21 days)'

        ELSE 'Very Slow (22+ days)'

    END as processing_category,

    COUNT(*) as claim_count,

    ROUND(AVG(processing_time), 2) as avg_days,

    ROUND(SUM(claim_amount), 2) as total_amount

FROM insurance.analytics.insurance_claims

GROUP BY 

    CASE 

        WHEN processing_time <= 7 THEN 'Fast (1-7 days)'

        WHEN processing_time <= 14 THEN 'Normal (8-14 days)'

        WHEN processing_time <= 21 THEN 'Slow (15-21 days)'

        ELSE 'Very Slow (22+ days)'

    END

ORDER BY avg_days

""")



processing_efficiency.show()


# Monthly claims trends

print("\n=== Monthly Claims Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(claim_date, 'yyyy-MM') as month,

       COUNT(*) as total_claims,

       ROUND(SUM(claim_amount), 2) as monthly_payout,

       ROUND(AVG(claim_amount), 2) as avg_claim_amount,

       ROUND(AVG(risk_score), 2) as avg_risk_score,

       COUNT(DISTINCT customer_id) as unique_claimants

FROM insurance.analytics.insurance_claims

GROUP BY DATE_FORMAT(claim_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Customer Risk Analysis ===


+-----------+------------+-------------+----------------+--------------+-------------------+
|customer_id|total_claims|total_claimed|avg_claim_amount|avg_risk_score|avg_processing_days|
+-----------+------------+-------------+----------------+--------------+-------------------+
| CUST007870|          12|    436160.35|         36346.7|          55.5|               38.5|
| CUST002884|          12|     430604.8|        35883.73|         54.83|              33.58|
| CUST002783|          10|    424762.01|         42476.2|          60.5|               29.1|
| CUST006960|          11|    418878.93|         38079.9|         69.91|              48.64|
| CUST001729|          11|    412611.37|        37510.12|         67.55|              37.18|
| CUST000883|          12|    395490.98|        32957.58|         60.58|              34.75|
| CUST004078|          12|    395238.21|        32936.52|         76.92|              27.42|
| CUST003279|          12|    389299.34|        32441.61|         69.9

+-----------+------------+-------------+----------------+-------------------+------------------+
|policy_type|total_claims| total_payout|avg_claim_amount|avg_processing_days|affected_customers|
+-----------+------------+-------------+----------------+-------------------+------------------+
|     Health|        7136|6.025945934E7|         8444.43|               14.5|              1341|
|       Life|        1473|5.693053767E7|        38649.38|              37.63|              1434|
|   Property|        1733|3.911063925E7|        22568.17|              22.22|              1445|
|       Auto|        3150|2.241207207E7|         7114.94|              17.87|              1541|
|       Home|        1508|1.988170738E7|        13184.16|              25.84|              1440|
+-----------+------------+-------------+----------------+-------------------+------------------+


=== Claims Processing Efficiency ===


+--------------------+-----------+--------+--------------+
| processing_category|claim_count|avg_days|  total_amount|
+--------------------+-----------+--------+--------------+
|     Fast (1-7 days)|       2110|    5.35|    4533004.37|
|  Normal (8-14 days)|       4624|   10.85| 2.668768244E7|
|   Slow (15-21 days)|       2623|    18.0| 4.000567373E7|
|Very Slow (22+ days)|       5643|    32.6|1.2736805517E8|
+--------------------+-----------+--------+--------------+


=== Monthly Claims Trends ===


+-------+------------+--------------+----------------+--------------+----------------+
|  month|total_claims|monthly_payout|avg_claim_amount|avg_risk_score|unique_claimants|
+-------+------------+--------------+----------------+--------------+----------------+
|2024-01|        1303| 1.700282262E7|        13048.98|         58.71|            1077|
|2024-02|        1159| 1.530272761E7|        13203.39|         58.93|             966|
|2024-03|        1290| 1.739153802E7|        13481.81|         58.84|            1074|
|2024-04|        1206|  1.61826177E7|        13418.42|         59.45|            1007|
|2024-05|        1324|  1.80161403E7|        13607.36|         59.54|            1086|
|2024-06|        1220|  1.56975657E7|        12866.86|          58.2|            1008|
|2024-07|        1309| 1.737609287E7|        13274.33|         57.87|            1081|
|2024-08|        1222| 1.543843201E7|        12633.74|          57.6|            1029|
|2024-09|        1292| 1.686660742E7|      

## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (customer_id, claim_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (customer_id, claim_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Insurance analytics where claims processing and risk assessment are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for insurance data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles insurance-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger insurance datasets
- Integrate with real claims processing systems

This notebook demonstrates how Oracle AI Data Platform makes advanced insurance analytics accessible while maintaining enterprise-grade performance and governance.